In [1]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2
from math import floor
from math import ceil
from sklearn.svm import LinearSVC
import operator
from functools import reduce
import itertools
import random
import os
import time

from Utilities.G2Dependency import *
from Utilities.ClassicMM import *
import copy
from ipynb.fs.defs.RandomWalk import RandomWalk
#from ipynb.fs.defs.GraphReader import GraphReader
from random import shuffle

In [2]:
X = pd.read_csv("/home/a20114261/sdelrio/GeneInteractions/GeneInteractionsBN_Datasets/Balanced_Batches/censored_CTNNB1/batch_0.csv",delimiter=',', engine='python')

In [3]:
# Data transformation
X['Pvalue'] = list(map(lambda x: 1 if x<=0.01 else 0,list(X['Pvalue'])))
del X['CauseGene']
del X['EffectGene']
del X['Replicate']
del X['Treatment']

In [4]:
X.head()

,Pvalue,C0_C15~E0_E15,C15_C30~E15_E30,C30_C90~E30_E90,C90_C120~E90_E120,C120_C180~E120_E180,C180_C210~E180_E210,C210_C240~E210_E240,C240_C360~E240_E360,C0_C15~E15_E30,...,C0_C240~E15_E360,C0_C240~E0_E360,C0_C360~E240_E360,C0_C360~E210_E360,C0_C360~E180_E360,C0_C360~E120_E360,C0_C360~E90_E360,C0_C360~E30_E360,C0_C360~E15_E360,C0_C360~E0_E360
0,1,1,1,0,1,-1,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,1,-1,-1,-1,1,1,-1,-1,1,1,...,-1,1,-1,-1,0,1,0,-1,-1,1
2,1,0,0,1,0,1,0,0,0,0,...,1,0,0,1,0,-1,-1,-1,1,0
3,1,0,-1,-1,0,0,0,-1,1,0,...,-1,-1,1,-1,1,1,1,1,1,1
4,1,1,1,0,-1,1,1,1,1,-1,...,-1,-1,1,-1,1,0,1,0,1,1


In [5]:
class MMRWPC():
    def __init__(self, alpha, delta, max_neighbours):
        
        self._RW = RandomWalk(alpha, delta, max_neighbours)
        self._alpha = alpha
        self._G2Dep = None
        self._arrX= None
        
        return
    
    def recieveX(self, X):
        self._arrX = copy.deepcopy(X)
        return
    
    def fit(self, X):
        """
        Fit training dataset X, completing G2Dep matrix and training RW
        """
        self._RW.fit(X)
        self._G2Dep = self._RW.get_G2Dep()
        self._arrX = arrayX(X)
        self._rw_iter = int(100*np.log(len(self._arrX))/2)
        return
    
    def __pc_len(self, CPC):
        counter = 0
        for n_set in CPC:
            counter += len(n_set)
        return counter
    
    def evaluate(self, index):
        
        start = time.clock()
        CPC=[]
        alpha = self._alpha
        X = self._arrX
        TargetNode = [x for x in self._arrX if x['name'].split('@')[0] == ("Node"+str(index+1))][0]
        Universe = arrayUniverse(TargetNode['name'], self._arrX)
        print("Entering Phase I")
        print("MMPC_beggining: \n"+str(len(Universe)))
        while len(Universe)>0:
            CPC_old = list(CPC) # copy
            maxminheur=self.__mm_heuristic_v3(TargetNode,CPC,list(Universe),X,alpha)
            F_arr = maxminheur[0]
            if (len(F_arr)>0):
                max_assoc = max([x[1] for x in F_arr])
            else:
                max_assoc = 0
            Universe = maxminheur[1]
            if max_assoc > 0:
                # append all nodes that fulfill the requirements
                newly_added_nodes = []
                for nn in [x[0] for x in F_arr if x[1]>=max_assoc]:
                    newly_added_nodes.append(nn)
                    indF=Universe.index(nn)
                    Universe.pop(indF)
                if (len(newly_added_nodes)>0):
                    CPC.append(newly_added_nodes)
            
            if (self.__pc_len(CPC)==self.__pc_len(CPC_old)):
                break
            print("\nUniverse actual size:")
            print(len(Universe))
            print("CPC size before filtering: "+str(self.__pc_len(CPC)))
            # Embedded RW in backward_phase
            CPC = self.__rw_community_selection_phase_v2(TargetNode,
                                                         list(itertools.chain.from_iterable(CPC))
                                                         ,X,alpha,self._rw_iter)
            print("CPC size after filtering: "+str(self.__pc_len(CPC)))
            for n_set in CPC: 
                print(list(map(lambda x:x['name'],n_set)))
        
        stop = time.clock()
        results = {}
        results['CPC'] = CPC
        results['time'] = stop - start
        return results
        

In [6]:
class MMRWPC(MMRWPC):
    
    def __mm_heuristic(self, TargetNode, CPC, Universe, X, alpha):
        F_arr=[]
        Z = CPC
        fixedCondVars = []
        if (len(CPC)>0):
            Z = CPC[0:-1]           # all but the last one   
            fixedCondVars = [CPC[-1]] # we use last one
        for i in range(len(Universe)-1,-1,-1):
            if (len(Universe[i])==0):
                continue
            assoc = self.__min_assoc(TargetNode,Universe[i],Z,fixedCondVars,X,alpha)
            #assoc = self.__min_assoc_v2(TargetNode,Universe[i],Z,fixedCondVars,X,alpha,10000)
            if (assoc>0):
                F_arr.append([Universe[i],assoc])
            if (assoc==0):
                Universe.pop(i)

        return [F_arr,Universe]
    
    def __mm_heuristic_v3(self, TargetNode, CPC, Universe, X, alpha):
        F_arr=[]
        Z = CPC
        if (len(CPC)>0):
            Z = CPC   
        for i in range(len(Universe)-1,-1,-1):
            if (len(Universe[i])==0):
                continue
            assoc = self.__min_assoc_v3(TargetNode,Universe[i],Z,X,alpha)
            #assoc = self.__min_assoc_v2(TargetNode,Universe[i],Z,fixedCondVars,X,alpha,10000)
            if (assoc>0):
                F_arr.append([Universe[i],assoc])
            if (assoc==0):
                Universe.pop(i)

        return [F_arr,Universe]
    
    
    def __min_assoc_v3(self, TargetNode, Xi, super_Z, X, alpha):
    
        min_assoc=999
        if self.__pc_len(super_Z)==0:
            min_assoc = self._G2Dep.dependency(TargetNode, Xi, [], alpha)        
        elif self.__pc_len(super_Z)==1:
            min_assoc = self._G2Dep.dependency(TargetNode, Xi, super_Z[0], alpha)
        else:
            for Z in super_Z:
                for IDsubsetZ_dec in range(0,2**len(Z)-1):
                    IDsubsetZ_bin = bin(IDsubsetZ_dec)
                    subsetZ = getZsubset(IDsubsetZ_bin,Z)            
                    subsetZ_assoc=self._G2Dep.dependency(TargetNode, Xi, subsetZ,alpha)
                    #counter+=1
                    #print(subsetZ_assoc[IDsubsetZ_dec])
                    if subsetZ_assoc < min_assoc:
                        min_assoc = subsetZ_assoc
                        if (min_assoc==0):
                            break
        return min_assoc
    
    def __min_assoc(self, TargetNode, Xi,Z, fixedCondVars, X, alpha):
    
        min_assoc=999
        if len(Z)==0:
            min_assoc = self._G2Dep.dependency(TargetNode, Xi, fixedCondVars, alpha)
            subsetZ_min_assoc = fixedCondVars        
        else:
            #print(2**len(Z)-1)
            for IDsubsetZ_dec in range(0,2**len(Z)-1):
                IDsubsetZ_bin = bin(IDsubsetZ_dec)
                subsetZ = getZsubset(IDsubsetZ_bin,Z)            
                subsetZ_assoc=self._G2Dep.dependency(TargetNode, Xi, fixedCondVars+subsetZ,alpha)
                #counter+=1
                #print(subsetZ_assoc[IDsubsetZ_dec])
                if subsetZ_assoc < min_assoc:
                    min_assoc = subsetZ_assoc
                    subsetZ_min_assoc = fixedCondVars + subsetZ
                    if (min_assoc==0):
                        break
        return min_assoc
    
    def __rw_node_all(self, t_index, t_iter, threshold, CPC):
        return [CPC,[]]
    
    def __rw_node_v2(self, t_index, t_iter, CPC):
        # will work at name level, then reconstruct list
        cpc_names = [x['name'].split('@')[0] for x in CPC]
        evaluation = self._RW.evaluate_node_index(t_index,t_iter)
        results = [x for x in sorted(evaluation["Node"+str(t_index+1)].items(), key=lambda x:x[1], reverse=True)]
        cpc_results = []
        g_ratio = (1+5**(1/2))/2
        set_results = [x[0] for x in results if x[0] in cpc_names and (x[1])>t_iter*g_ratio/(len(self._arrX))]
        
        set_returnable = []
        for i in range(len(CPC)-1,-1,-1):
            if CPC[i]['name'].split('@')[0] in set_results:
                set_returnable.append(CPC.pop(i))
        #print([x['name'] for x in CPC])
        return [set_returnable, CPC]
    
    def __rw_node(self, t_index, t_iter, threshold, CPC):
        # will work at name level, then reconstruct list
        cpc_names = [x['name'].split('@')[0] for x in CPC]
        evaluation = self._RW.evaluate_node_index(t_index,t_iter)
        results = [x[0] for x in sorted(evaluation["Node"+str(t_index+1)].items(), key=lambda x:x[1], reverse=True)]
        cpc_results = []
        set_results = [x for x in results if x in cpc_names][:threshold-1]
        
        set_returnable = []
        for i in range(len(CPC)-1,-1,-1):
            if CPC[i]['name'].split('@')[0] in set_results:
                set_returnable.append(CPC.pop(i))
        
        return [set_returnable, CPC]
    
    
    
    def __rw_community_selection_phase(self, TargetNode, CPC, X, alpha, n_iter):
        votation_arr = []
        if (len(CPC)>1):
            # will divide the sets in np.log(size)*2
            CPC = sorted(CPC,key=lambda x:self._G2Dep.dependency(TargetNode,x,[],alpha),reverse = True)
            size_sets = ceil(np.log(len(CPC))*2)
            print("Size set for community selection phase will be: "+str(size_sets))
            n_sets = ceil(len(CPC)/size_sets)
            CPC_copy = list(CPC)
            nodes_sets = []
            if (len(CPC)>5):
                for set_counter in range(0,n_sets):
                    rand_node = CPC_copy[0]
                    rand_true_index = int(CPC_copy[0]['name'].split('@')[0].split('Node')[1]) - 1
                    CPC_copy.pop(0)
                    rw_results = self.__rw_node(rand_true_index, n_iter, size_sets, CPC_copy)
                    node_set = [rand_node] + rw_results[0]
                    CPC_copy = rw_results[1]
                    nodes_sets.append(node_set)
            else:
                nodes_sets.append(CPC)
            pre_cpc = []
            for n_set in nodes_sets:
                for node in self.__classic_filtering_phase(TargetNode, n_set, X, alpha):
                    pre_cpc.append(node)
            return pre_cpc
        else:
            return CPC
        

        
    
    def __rw_community_selection_phase_v2(self, TargetNode, CPC, X, alpha, n_iter):
        votation_arr = []
        if (len(CPC)>1):
            # will divide the sets in np.log(size)*2
            CPC = sorted(CPC,key=lambda x:self._G2Dep.dependency(TargetNode,x,[],alpha),reverse = True)
            CPC_copy = list(CPC)
            nodes_sets = []
            if (len(CPC)>5):
                while(len(CPC_copy)>0):
                    if len(CPC_copy) == 1:
                        nodes_sets.append([CPC_copy.pop(0)])
                        break
                        
                    rand_index = random.randint(0,len(CPC_copy))-1                    
                    rand_true_index = int(CPC_copy[rand_index]['name'].split('@')[0].split('Node')[1]) - 1
                    rand_node = CPC_copy.pop(rand_index)
                    rw_results = self.__rw_node_v2(rand_true_index, n_iter, CPC_copy)
                    node_set = [rand_node] + rw_results[0]
                    CPC_copy = rw_results[1]
                    nodes_sets.append(node_set)
            else:
                nodes_sets.append(CPC)
            pre_cpc = []
            for n_set in nodes_sets:
                print([x['name'] for x in n_set])
                if (len(n_set)>1):
                    print("n_set of length: "+str(len(n_set)))
                    result_set = self.__classic_filtering_phase(TargetNode, n_set, X, alpha)
                    pre_cpc.append(result_set)
                else:
                    print("n_set of length: 1")
                    pre_cpc.append(n_set) 
                    continue
                    
            return pre_cpc
        else:
            return [CPC]
    
    
    
    def __community_selection_phase(self, TargetNode, CPC, X, alpha, loops_flag):
        votation_arr = []
        if (len(CPC)>1):
            
            # will divide the sets in np.log(size)*2
            size_sets = ceil(np.log(len(CPC))*2)
            print("Size set for community selection phase will be: "+str(size_sets))
            n_sets = ceil(len(CPC)/size_sets)
            for k in range(0,loops_flag):
                nodes_sets = []
                CPC_copy = list(CPC)
                for set_counter in range(0,n_sets):
                    sub_node_set = []
                    rand_index = random.randint(0,len(CPC_copy))-1
                    rand_node = CPC_copy[rand_index]
                    rand_true_index = int(CPC_copy[rand_index]['name'].split('@')[0].split('Node')[1]) - 1
                    CPC_copy.pop(rand_index)
                    rw_results = self.__rw_node(rand_true_index, 1, size_sets, CPC_copy)
                    sub_node_set = [rand_node] + rw_results[0]
                    print(len(sub_node_set))
                    CPC_copy = rw_results[1]
                    nodes_sets.append(sub_node_set)
                pre_cpc = []
                for n_set in nodes_sets:
                    for node in self.__classic_filtering_phase(TargetNode, n_set, X, alpha):
                        pre_cpc.append(node)
#                     for node in self.__filtering_phase(TargetNode, n_set, X, alpha):
#                         pre_cpc.append(node)
                votation_arr.append(pre_cpc)
            
            # voting
            returnable_cpc = []
            for node in CPC:
                v_counter=0
                for votation in votation_arr:
                    if node in votation:
                        v_counter=v_counter+1
                if (v_counter>(loops_flag-loops_flag**(1/2))):
                    returnable_cpc.append(node)
            return returnable_cpc
        else:
            return CPC

In [7]:
class MMRWPC(MMRWPC):
    
    def __classic_filtering_phase(self, TargetNode, CPC, X, alpha):
        filtered_cpc = []
        if (len(CPC)>1):
            for i in range(len(CPC)-1,-1,-1):
                evaluated_node = CPC.pop(i)
                if self.__exist_dseparator(TargetNode,evaluated_node,CPC,X,alpha)[0] == False:
                    filtered_cpc.append(evaluated_node)
        else:
            return CPC
        return filtered_cpc
    
    def __filtering_phase(self, TargetNode, CPC, X,alpha):
        #print("\nEntering Phase II")
        if (len(CPC)>1):
            # will divide the sets in 
            Z=list(CPC)
            while len(Z)>0:
                # Will test all vs the Target, and select the lowest
                minDep=999
                minDepIndex=-1
                for i in range(0,len(Z)):
                    zCopy = list(Z)
                    zCopy.pop(i)
                    auxDep = self._G2Dep.dependency(TargetNode,Z[i],zCopy,alpha)
                    if (minDep>auxDep):
                        minDep = auxDep
                        minDepIndex=i
                # Find index in CPC
                CPCindex=-1
                for i in range(0,len(CPC)):
                    if (CPC[i]['name']==Z[minDepIndex]['name']):
                        CPCindex=i
                        break

                # Remove from Z
                Z.pop(minDepIndex)
                #print_names(Z)
                #print("Analyzing D-separator for ",CPC[CPCindex]['name'])
                if self.__exist_dseparator(TargetNode,CPC[CPCindex],Z,X,alpha)[0] == True:
                    #print("it did exist! removing from cpc.")
                    CPC.pop(CPCindex)
        return CPC
    
    def __exist_dseparator(self, TargetNode,Xi, Z, X, alpha):
        flagExist = False
        dsepSet=[]
        #counter=0
        #print_names(Z)
        for i in range(0,(2**len(Z))-1):
            IDsubsetZ_dec = i
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            # no cache
            #print("from exist dseparator")
            dep = self._G2Dep.dependency(TargetNode,Xi,subsetZ, alpha)
            #print(subsetZ)
            #print(dep)
            if (dep==0):
                flagExist = True
                dsepSet = subsetZ
                break
        #print("Module exist d-separator: ",counter)
        return [flagExist,dsepSet]
    
    def shuffle_rows(self):
        shuffle(self._arrX)
        
        return

In [8]:
MMPC_RW_model = MMRWPC(0.05,0.0,1)

Random Walk successful instantiation with:
-Alpha: 0.05
-Delta: 0.0
-Max Neighbours per run: 1


In [9]:
#%%time
MMPC_RW_model.fit(X)

Succesful instantiation of Dep object
Succesfully fitted array of 541 columns
and 6000 rows per column.


In [10]:
#%%time
result = MMPC_RW_model.evaluate(0)

Entering Phase I
MMPC_beggining: 
540

Universe actual size:
188
CPC size before filtering: 1
CPC size after filtering: 1
['Node162@2']

Universe actual size:
173
CPC size before filtering: 2
['Node162@2', 'Node428@2']
n_set of length: 2
CPC size after filtering: 2
['Node428@2', 'Node162@2']

Universe actual size:
149
CPC size before filtering: 3
['Node162@2', 'Node243@2', 'Node428@2']
n_set of length: 3
CPC size after filtering: 3
['Node428@2', 'Node243@2', 'Node162@2']

Universe actual size:
71
CPC size before filtering: 4
['Node162@2', 'Node243@2', 'Node78@2', 'Node428@2']
n_set of length: 4
CPC size after filtering: 3
['Node78@2', 'Node243@2', 'Node162@2']

Universe actual size:
47
CPC size before filtering: 4
['Node162@2', 'Node243@2', 'Node338@2', 'Node78@2']
n_set of length: 4
CPC size after filtering: 4
['Node78@2', 'Node338@2', 'Node243@2', 'Node162@2']

Universe actual size:
22
CPC size before filtering: 5
['Node162@2', 'Node243@2', 'Node338@2', 'Node78@2', 'Node479@2']
n_set

In [40]:
[x['name'] for x in itertools.chain.from_iterable(result['CPC'])][0].split('@')[0].split('Node')[1]

'162'

In [8]:
graph_reader = GraphReader('/home/sdelrio/Alarm10_graph.txt')

In [273]:
graph_reader.getPC(94-16)

['Node80', 'Node87']

In [277]:
#%%capture
t_index = 30
n_loops = 100
results = {
    'distance':[],
    'time':[]
}
# MMPC_RW_model.shuffle_rows()
# CPC = MMPC_RW_model.evaluate(t_index)
# CPC['CPC'] = list(itertools.chain.from_iterable(CPC['CPC']))
# print(graph_reader.getDistance(t_index,[x['name'].split('@')[0] for x in CPC['CPC']])['distance-pc'])

for i in range(0,n_loops):
    for k in range(0,10):
        MMPC_RW_model.shuffle_rows()
    CPC = MMPC_RW_model.evaluate(t_index)
    CPC['CPC'] = list(itertools.chain.from_iterable(CPC['CPC']))
    results['distance'].append(graph_reader.getDistance(t_index,[x['name'].split('@')[0] for x in CPC['CPC']])['distance-pc'])
    results['time'].append(CPC['time'])

In [278]:
print('mean time:')
mean_time =  sum(results['time'])/n_loops
print(mean_time)
print('max time:')
print(max(results['time']))
print('min time:')
print(min(results['time']))
print('standard dev:')
print((sum(list(map(lambda x:x**2-mean_time**2,results['time'])))/n_loops)**(1/2))
print('mean distance:')
mean_distance =  sum(results['distance'])/n_loops
print(mean_distance)
print('max distance:')
print(max(results['distance']))
print('min distance:')
print(min(results['distance']))
print('standard dev:')
print((sum(list(map(lambda x:x**2-mean_distance**2,results['distance'])))/n_loops)**(1/2))

mean time:
0.06180558999985806
max time:
0.12620400000014342
min time:
0.057942999999795575
standard dev:
0.007520364767816502
mean distance:
0.0
max distance:
0.0
min distance:
0.0
standard dev:
0.0


In [ ]:
# #%%capture
# gmean_results=[]
# for i in range(0, 370):
#     index_t_node = i
#     CPC = MMPC_RW_model.evaluate(index_t_node)
#     #print([x['name'] for x in CPC])
#     gmean_results.append(['Node'+str(i+1),graph_reader.getGmean(index_t_node, [x['name'].split('@')[0] for x in CPC])])
    
# print(gmean_results)
# # print("Beginning CPC of each PC node")
# # relevant_sets = []
# # for e in CPC:
# #     relevant_sets.append(MMPC_RW_model.evaluate(int(e['name'].split('@')[0].split('Node')[1])-1))

In [123]:
# for e in sorted(gmean_results,key=lambda x:x[1]['gmean-pc']):
#     print(e)

['Node24', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node135', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node205', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node320', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node357', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node27', {'gmean-mb': 0.14088941361668633, 'gmean-pc': 0.2465846994535519}]
['Node363', {'gmean-mb': 0.2, 'gmean-pc': 0.25}]
['Node128', {'gmean-mb': 0.3315168029064487, 'gmean-pc': 0.3315168029064487}]
['Node353', {'gmean-mb': 0.24863387978142076, 'gmean-pc': 0.3315168029064487}]
['Node30', {'gmean-mb': 0.19945205479452055, 'gmean-pc': 0.332425068119891}]
['Node104', {'gmean-mb': 0.19945205479452055, 'gmean-pc': 0.332425068119891}]
['Node172', {'gmean-mb': 0.24931693989071038, 'gmean-pc': 0.332425068119891}]
['Node215', {'gmean-mb': 0.24931693989071038, 'gmean-pc': 0.332425068119891}]
['Node20', {'gmean-mb': 0.24385245901639344, 'gmean-pc': 0.48777173913043476}]
['Node248', {'gmean-mb': 0.3260672116257947, 'gmean-pc': 0.4891304347826087}]
['Node102

In [126]:
# save_folder = '/home/sdelrio/alarm_datasets/log_results/'
# preselected_nodes=[] # will have [node_str,balance_ranking]
# # so far, we need Node, size of estimated pc set per filename, balance ranking
# ff = open(save_folder+'mmpc_partitioned/alarm10_binary_nodes_selected.txt','r')
# for e in ff.read().split('\n'):
#     spl_line = e.split(';')
#     if (len(spl_line[0])==0):
#         continue
#     preselected_nodes.append([spl_line[0],float(spl_line[1])])
# ff.close()

In [129]:
# pre_nodes_names = [x[0].split('@')[0] for x in preselected_nodes]

In [131]:
# for e in sorted([x for x in gmean_results if x[0] in pre_nodes_names],key=lambda x:x[1]['gmean-pc']):
#     print(e)

['Node24', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node135', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node205', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node320', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node357', {'gmean-mb': 0.0, 'gmean-pc': 0.0}]
['Node27', {'gmean-mb': 0.14088941361668633, 'gmean-pc': 0.2465846994535519}]
['Node363', {'gmean-mb': 0.2, 'gmean-pc': 0.25}]
['Node128', {'gmean-mb': 0.3315168029064487, 'gmean-pc': 0.3315168029064487}]
['Node353', {'gmean-mb': 0.24863387978142076, 'gmean-pc': 0.3315168029064487}]
['Node30', {'gmean-mb': 0.19945205479452055, 'gmean-pc': 0.332425068119891}]
['Node104', {'gmean-mb': 0.19945205479452055, 'gmean-pc': 0.332425068119891}]
['Node172', {'gmean-mb': 0.24931693989071038, 'gmean-pc': 0.332425068119891}]
['Node215', {'gmean-mb': 0.24931693989071038, 'gmean-pc': 0.332425068119891}]
['Node20', {'gmean-mb': 0.24385245901639344, 'gmean-pc': 0.48777173913043476}]
['Node102', {'gmean-mb': 0.32692307692307687, 'gmean-pc': 0.49043715846994534}]
['Node2

In [135]:
# 1/1.6180339887

0.618033988768953

In [220]:
# z = [1,2,3,4]

In [222]:
# import copy
# copy.deepcopy(z)

[1, 2, 3, 4]

In [26]:


list_of_lists = [[1,2,3],[4,5,6]]
list(itertools.chain.from_iterable(list_of_lists))

[1, 2, 3, 4, 5, 6]

In [54]:
list(map(lambda x:x+1,[1,2,3]))

[2, 3, 4]

In [147]:
int(100*np.log(370)/2)

295